In [1]:
import polars as pl

url = "https://huggingface.co/datasets/pythainlp/thai_food_v1.0/resolve/main/data/train-00000-of-00001-e8b362f32bb3715c.parquet"
df = pl.read_parquet(url)
with pl.Config() as cfg:
    cfg.set_fmt_str_lengths(100)
    display(df.sample(3))
corpus = df["text"].to_list()

name,text
str,str
"""ปลาทูร่องสวน""","""# ปลาทูร่องสวน ## เครื่องปรุง - ปลาทูนึ่งขนาดกลาง 3 ตัว - ผักบุ้งเก็บในสวนใช้แต่ยอด 3 กำมือ - ม…"
"""ไก่ยำ""","""# ไก่ยำ ## เครื่องปรุง - อกไก่ 4 อก - แป้งสาลีนิดหน่อย - หอมหัวเล็ก หั่นละเอียด 1 ช้อนหวาน - ห…"
"""ยำไข่แมงดา""","""# ยำไข่แมงดา ## เครื่องปรุง - ไข่แมงดานึ่งสุกแล้ว 1/2 ถ้วยชา - กุ้งนาง 3 ตัว - หอมเผา 5 หัว - …"


In [2]:
import regex

UNICODE_THAI_CHAR = r"\u0E01-\u0E2F"  # ตัวพยัญชนะ
UNICODE_THAI_VOWEL = r"\u0E30-\u0E39\u0E40-\u0E47\u0E4C-\u0E4E"  # สระ
UNICODE_THAI_INTONATION = r"\u0E48-\u0E4B"  # วรรณยุกต์
UNICODE_LETTERS = r"\p{L}"  # from any language
UNICODE_DIGITS = r"\p{N}"

MATCH_ALL_NON_LETTERS = regex.compile(
    rf"^[^{UNICODE_LETTERS}{UNICODE_DIGITS}{UNICODE_THAI_CHAR}{UNICODE_THAI_VOWEL}{UNICODE_THAI_INTONATION}]+$"
)

# Test cases
test_cases = [
    # Should MATCH (only non-letters/non-digits)
    ("!!!", True, "punctuation only"),
    ("...", True, "periods only"),
    ("   ", True, "whitespace only"),
    ("@#$%", True, "special chars only"),
    ("---", True, "hyphens only"),
    ("()", True, "brackets only"),
    ("_", True, "underscore only"),
    ("_ _", True, "underscores and spaces"),
    # Should NOT match (contains letters or digits)
    ("hello", False, "English letters"),
    ("สวัสดี", False, "Thai letters"),
    ("ก่อน", False, "Thai with tone marks"),
    ("123", False, "digits"),
    ("test_123", False, "letters and digits"),
    ("你好", False, "Chinese characters"),
    ("مرحبا", False, "Arabic"),
    ("hello!", False, "letters with punctuation"),
    ("ข้าว", False, "Thai with vowel marks"),
]

print("Testing MATCH_ALL_NON_LETTERS pattern:")
print("=" * 60)
for text, expected_match, description in test_cases:
    result = MATCH_ALL_NON_LETTERS.match(text)
    matched = result is not None
    status = "✓" if matched == expected_match else "✗"
    match_str = "MATCH" if matched else "NO MATCH"
    print(f"{status} '{text:15}' -> {match_str:10} ({description})")

Testing MATCH_ALL_NON_LETTERS pattern:
✓ '!!!            ' -> MATCH      (punctuation only)
✓ '...            ' -> MATCH      (periods only)
✓ '               ' -> MATCH      (whitespace only)
✓ '@#$%           ' -> MATCH      (special chars only)
✓ '---            ' -> MATCH      (hyphens only)
✓ '()             ' -> MATCH      (brackets only)
✓ '_              ' -> MATCH      (underscore only)
✓ '_ _            ' -> MATCH      (underscores and spaces)
✓ 'hello          ' -> NO MATCH   (English letters)
✓ 'สวัสดี         ' -> NO MATCH   (Thai letters)
✓ 'ก่อน           ' -> NO MATCH   (Thai with tone marks)
✓ '123            ' -> NO MATCH   (digits)
✓ 'test_123       ' -> NO MATCH   (letters and digits)
✓ '你好             ' -> NO MATCH   (Chinese characters)
✓ 'مرحبا          ' -> NO MATCH   (Arabic)
✓ 'hello!         ' -> NO MATCH   (letters with punctuation)
✓ 'ข้าว           ' -> NO MATCH   (Thai with vowel marks)


In [3]:
from collections import Counter
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

THAI_STOPWORDS = thai_stopwords()
def preprocess_text(text: str):
    tokenzied_text = word_tokenize(text)
    tokenzied_text = filter(lambda word: word not in THAI_STOPWORDS, tokenzied_text)
    tokenzied_text = filter(lambda word: not MATCH_ALL_NON_LETTERS.match(word), tokenzied_text)
    tokenzied_text = filter(lambda word: word != "", tokenzied_text)
    return list(tokenzied_text)


def create_vocaburaries(corpus: list[str], return_list: bool = False) -> Counter:
    vocab = Counter()
    for doc in corpus:
        tokenzied_doc = preprocess_text(doc)
        vocab.update(tokenzied_doc)

    if return_list:
        return list(vocab.keys())
    return vocab

In [4]:
vocab = create_vocaburaries(corpus)
print(f"Vocab size: {len(vocab)}")
vocab.most_common(10)

Vocab size: 1786


[('ใส่', 765),
 ('1', 651),
 ('หั่น', 391),
 ('2', 307),
 ('น้ำ', 239),
 ('ถ้วย', 236),
 ('ตัก', 231),
 ('ช้อนโต๊ะ', 230),
 ('หอม', 192),
 ('เครื่องปรุง', 190)]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(
    tokenizer=preprocess_text,
    lowercase=False,
    token_pattern=None,
)
corpus_count_vec = count_vectorizer.fit_transform(corpus)

In [6]:
corpus_count_vec.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], shape=(159, 1786))

In [7]:
count_vectorizer.vocabulary_

{'กุ้ง': 161,
 'ทา': 604,
 'พริกไทย': 867,
 'กระเทียม': 94,
 'เครื่องปรุง': 1380,
 'กุ้งนาง': 163,
 '4': 42,
 'ตัว': 501,
 '5': 45,
 'เม็ด': 1507,
 'กลีบ': 112,
 '2': 33,
 'ราก': 1011,
 'ผักชี': 817,
 'น้ำปลา': 688,
 'ช้อนโต๊ะ': 412,
 'น้ำมันหมู': 696,
 '1': 20,
 'วิธีทำ': 1116,
 'ล้าง': 1092,
 'สะอาด': 1151,
 'ปอก': 779,
 'ผ่า': 843,
 'หาง': 1283,
 'ลึก': 1078,
 'มีด': 947,
 'เบาะ': 1462,
 'ตามขวาง': 512,
 'เอ็น': 1581,
 'เวลา': 1531,
 'ทอด': 592,
 'งอ': 335,
 'โขลก': 1674,
 'ละเอียด': 1062,
 'เคล้า': 1387,
 'กะทะ': 127,
 'ตั้งไฟ': 506,
 'ใส่': 1724,
 'น้ำมัน': 693,
 'ร้อน': 1035,
 'สุก': 1197,
 'ตัก': 495,
 'จาน': 349,
 'ราดหน้า': 1015,
 'ข้าวเม่าทอด': 254,
 'กล้วยไข่': 120,
 'หวี': 1265,
 '(1': 0,
 'ชี': 394,
 'ก': 56,
 'ค': 259,
 'ยา': 970,
 'ศารท': 1122,
 '/2': 16,
 'กิโลกรัม': 153,
 'ไข่': 1739,
 'เป็ด': 1481,
 'ฟอง': 897,
 'แป้ง': 1638,
 'ข้าวจ้าว': 242,
 'แห้ง': 1669,
 '3': 39,
 '/4': 18,
 'ถ้วย': 576,
 'ชา': 384,
 'น้ำ': 672,
 'ปูน': 801,
 'ใส': 1722,
 'เกลือ': 1347,
 'ช้อนชา'

In [8]:
from text import BM25Transformer

bm25 = BM25Transformer(norm=None, use_idf=True, smooth_idf=False, sublinear_tf=False)
bm25.fit(corpus_count_vec)

,norm,None
,use_idf,True
,smooth_idf,False
,sublinear_tf,False
,k1,1.5
,b,0.75


In [9]:
bm25.idf_

array([4.15888308, 4.66970871, 4.15888308, ..., 4.15888308, 4.66970871,
       4.66970871], shape=(1786,))

In [10]:
corpus_bm25_vec = bm25.transform(corpus_count_vec)

In [11]:
corpus_bm25_vec.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [4.53685768, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]], shape=(159, 1786))

In [12]:
text = "บะหมี่"
vector = count_vectorizer.transform([text])
vector = bm25.transform(vector)
vector

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1 stored elements and shape (1, 1786)>

In [13]:
import numpy as np
n = 3
order = list(np.argsort(np.array(np.dot(corpus_bm25_vec, vector.T).todense()).flatten())[::-1][:n])

In [14]:
with pl.Config() as cfg:
    cfg.set_fmt_str_lengths(50)
    display(df[order])

name,text
str,str
"""บะหมี่สำเร็จ""","""# บะหมี่สำเร็จ ## เครื่องปรุง - เส้นบะหมี่แห้ง …"
"""บะหมี่ทรงเครื่อง""","""# บะหมี่ทรงเครื่อง ## เครื่องปรุง - เส้นบะหมี่ …"
"""น้ำพริกไข่เค็ม""","""# น้ำพริกไข่เค็ม ## เครื่องปรุง - ไข่เค็มต้มสุก…"
